<a href="https://colab.research.google.com/github/pranjalchaubey/Predicting-Stock-Direction/blob/master/src/Stock_Direction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stock Direction
In this notebook we are going to use Random Forests to predict the direction  
of stocks for the next trading day.  
In more technical terms, we are going to try and predict the _**Momentum**_  
of the stocks.  
Based on the momentum, we are going to give the following signals to the user  
1. Buy (_+ve momentum_)
2. Sell (_-ve momentum_)
3. Hold (_neutral_)  

This system will work on the daily closing prices and will generate the signals by predicting the momentum for the next trading day. 

##Setup The Colab Environment 
Install and Import the required libraries.  

In [7]:
# Install the yFinance library 
!pip install yfinance 

  Created wheel for yfinance: filename=yfinance-0.1.45-cp36-none-any.whl size=14652 sha256=ec9264a40ffd123c8275d9405c7dda0e951a2f3691dff307fd030661ffd3cfe1
  Stored in directory: /root/.cache/pip/wheels/0c/d1/df/aa9a7744a4ac353cc9a1f2c3aaea7c1f457fc49de4286f2d88
Successfully built yfinance


In [0]:
# yFinance will help us fetch the data for our dataset
import yfinance as yf

In [0]:
# Data Mining and plotting 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [0]:
# Classifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
# Other Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import os
import sys
import shutil
# if in Google Colaboratory
try:
    from google.colab import drive
except:
    pass

import warnings
warnings.filterwarnings("ignore")

In [0]:
# Configure Pandas Display Options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
# Since I am running this notebook on Colab, 
# Let's try and get some system information 
import platform
print('System Processor: ', platform.processor(), '\n')
!nvidia-smi

System Processor:  x86_64 

Sat Sep 28 12:20:00 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                

## Load the Data
We will extract the 5 year history of 10 stocks chosen by the user. 

<br/>Here, we will take the following stocks as an example
1. Facebook FB
2. Apple AAPL
3. Amazon AMZN
4. Netflix NFLX
5. Google GOOGL
6. Starbucks SBUX
7. Exxon Mobil XOM
8. Johnson & Johnson JNJ
9. Bank of America BAC
10. General Motors GM

<br/>In the actual application, stock selection will be done by the user. 

In [0]:
# List of stock tickers 
# this info will come from the user, perhaps in the form of a pickle file 
ticker_list = ['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOGL', 'SBUX', 'XOM', 'JNJ',\
               'BAC', 'GM']

In [0]:
def fetch_ticker_data(tickers): 
  """
  Fetch 5 years of historical data for the 'tickers', from Yahoo Finance.    

  Parameters
  ----------
  tickers : List   
    List of stocks chosen by the user.   
      
  Returns
  -------
  data : Dataframe 
    Pandas df with historical stock data time series. 
  """
